In [1]:
# master=k8s://127.0.0.1:16443
import sys
import pandas as pd

sys.path.append('..')
from etl import create_spark_session

In [2]:
spark = create_spark_session(local=True)
spark.sparkContext.setLogLevel("ERROR")
spark

In [3]:
df_songplays = spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/songplays.parquet').alias("songplays")
df_times = spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/times.parquet').alias("times")
df_artists = spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/artists.parquet').alias("artists")
df_users = spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/users.parquet').alias("users")
df_songs = spark.read.format('parquet').load('s3a://dutrajardim/udacity-dl-project/songs.parquet').alias("songs")

In [6]:
# artists.name, songplays.level, times.weekday
cube_level_weekday_name = \
    df_songplays \
        .join(df_times, on='start_time', how='left') \
        .join(df_artists, on='artist_id', how='left') \
        .fillna("unknown", subset=['name']) \
        .cube('level', 'weekday', 'name')

In [11]:
cube_level_weekday_name.count() \
    .write.format('parquet') \
    .mode('overwrite') \
    .save('s3a://dutrajardim/olap-cubes/level-weekday-artist.parquet')

In [41]:
from pyspark.sql.functions import col
cube_year_week_gender = \
    df_songplays.select(['start_time', 'user_id', 'song_id']) \
        .join(df_times.select(['start_time', 'week']), on='start_time', how='left') \
        .join(df_users.select(['user_id', 'gender']), on='user_id', how='left') \
        .join(df_songs.select(['song_id', 'year']), on='song_id', how='left') \
        .fillna({
            'gender': "unknown",
            'year': -1
        }) \
        .cube('year', 'week', 'gender')

In [42]:
cube_year_week_gender.count() \
    .write.format('parquet') \
    .mode('overwrite') \
    .save('s3a://dutrajardim/olap-cubes/year-week-gender.parquet')

In [10]:
olap = spark.read.format('parquet').load('s3a://dutrajardim/olap-cubes/level-weekday-artist.parquet')
olap.where('weekday IS NULL AND name IS NULL').toPandas()

,level,weekday,name,count
0,None,NaN,None,8106
1,free,NaN,None,1782
2,paid,NaN,None,6324


In [11]:
spark.stop()